# Knowledge Embedding with Pykeen
## 概要
- 知識グラフから知識グラフの埋め込みモデルを学習する．
- pykeenにあらかじめ入っているデータセットに関しては，この[GitHub](https://github.com/pykeen/benchmarking)のページを参考にハイパーパラメータを設定．
## 入力データ・パラメータ
- 知識グラフ埋め込みモデル名
- 知識グラフ
- ランダムシード（optional）
## 出力データ
- 各ランダムシードの知識グラフ
- メタデータ

## modules

In [1]:
import os
import shutil
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm
from pykeen.pipeline import pipeline
from pykeen.datasets import get_dataset

from util.databinder import DataBinder

/tmp/42775677.1.gpu/ipykernel_2448201/3913572063.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## functions

In [2]:
def convert_dtype(value):
    if isinstance(value, np.floating):
        return float(value)
    elif isinstance(value, np.integer):
        return int(value)
    else:
        return value

def delete_all_files_in_directory(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            os.remove(file_path)
        for dir in dirs:
            dir_path = os.path.join(root, dir)
            shutil.rmtree(dir_path)

def get_best_params(f_params:str, model_name:str, dataset_name:str):
    """
    最適なハイパーパラメータを取得する．
    TODO:
        洗練されていないので修正すること．
    """
    df_best_params = pd.read_pickle(f_params).reset_index()
    df_best_params = df_best_params[df_best_params['model'].isin([model_name])]
    df_best_params = df_best_params[df_best_params['dataset'].isin([dataset_name])]
    
    dict_args = {}
    print(f'len of df_best_param: {len(df_best_params)}')
    idx = df_best_params.index[0]
    
    dict_args['dataset'] = df_best_params.loc[idx,'dataset']
    
    dict_args['dataset_kwargs'] = {}
    for k, v in df_best_params.filter(regex='pipeline_config.pipeline.dataset_kwargs').loc[idx].items():
        if not np.isnan(v):
            dict_args['dataset_kwargs'][k.split('.')[-1]] = v

    dict_args['evaluator'] = df_best_params.loc[idx,'evaluator']

    dict_args['evaluator_kwargs'] = {}
    for k, v in df_best_params.filter(regex='pipeline_config.pipeline.evaluator_kwargs').loc[idx].items():
        if not np.isnan(v):
            dict_args['evaluator_kwargs'][k.split('.')[-1]] = v
    
    dict_args['model'] = df_best_params.loc[idx,'model']

    dict_args['loss'] = df_best_params.loc[idx, 'loss']

    dict_args['regularizer'] = df_best_params.loc[idx, 'regularizer']

    dict_args['optimizer'] = df_best_params.loc[idx, 'optimizer']

    dict_args['optimizer_kwargs'] = {}
    for k, v in df_best_params.filter(regex='pipeline_config.pipeline.optimizer_kwargs').loc[idx].items():
        if not np.isnan(v) and 'automatic_memory_optimization' not in k:
            dict_args['optimizer_kwargs'][k.split('.')[-1]] = convert_dtype(v)
    
    dict_args['model_kwargs'] = {}
    for k, v in df_best_params.filter(regex='pipeline_config.pipeline.model_kwargs').loc[idx].items():
        if not np.isnan(v) and 'automatic_memory_optimization' not in k:
            k = k.split('.')[-1]
            if k in ['output_channels', 'kernel_height', 'kernel_width']:
                v = int(v)    
            dict_args['model_kwargs'][k] = convert_dtype(v)

    dict_args['training_loop'] = df_best_params.loc[idx, 'training_loop']

    dict_args['training_kwargs'] = {}
    for k, v in df_best_params.filter(regex='pipeline_config.pipeline.training_kwargs').loc[idx].items():
        if not np.isnan(v):
            k = k.split('.')[-1]
            if k in ['batch_size', 'num_epochs']:
                v = int(v)

            if k not in ['label_smoothing']:
                dict_args['training_kwargs'][k.split('.')[-1]] = v

    return dict_args

## parameters

In [3]:
# for input
# ----------------------------------------------------
## a path to the data fra
f_params = '../benchmarking/df_best_param.pkl'
## a name of knowledge graph embedding model
model_name = 'distmult'
## a data set (knowledge graph)
dataset_name = 'fb15k237'
## a list of random seeds which should be unique for each other
list_random_seeds = [0]
## a number of epochs (if None, using best value)
num_epochs = None

# for output
# ----------------------------------------------------
## a direcory where learned model are saved
del_previous_result = True
output_name = 'try1'
dir_learned_model = f'./models/20240803/kge_{output_name}_{model_name}_{dataset_name}'

## preparation

In [4]:
if del_previous_result:
    delete_all_files_in_directory(dir_learned_model)
if not os.path.exists(dir_learned_model):
    os.mkdir(dir_learned_model)
db = DataBinder(dir_learned_model)

INFO:root:Create ./models/20240803/kge_try1_distmult_fb15k237/info.json


## main

In [5]:
dict_args = get_best_params(f_params, model_name, dataset_name)

len of df_best_param: 1


In [6]:
if num_epochs != None:
    dict_args['training_kwargs']['num_epochs'] = num_epochs

In [ ]:
for random_seed in list_random_seeds:

    print(20*'-')
    print(f'learn knoweldge graph embedding with random seed {random_seed}')

    pipeline_result = pipeline(
        dataset=dict_args['dataset'],
        dataset_kwargs = dict_args['dataset_kwargs'],
        evaluator=dict_args['evaluator'],
        evaluator_kwargs = dict_args['evaluator_kwargs'],
        loss = dict_args['loss'],
        model= dict_args['model'],
        model_kwargs = dict_args['model_kwargs'],    
        training_kwargs=dict_args['training_kwargs'],
        optimizer=dict_args['optimizer'],
        optimizer_kwargs=dict_args['optimizer_kwargs'],
        stopper='early',
        stopper_kwargs={'frequency':50, 'patience':2, 'relative_delta':0.002},
        random_seed=random_seed
    )

    pipeline_result.plot_losses()

    hits_at_10 = pipeline_result.get_metric('hits_at_10')
    print(20*'-')
    print('RESULT\t' + str(hits_at_10))

    dir_save = f'{dir_learned_model}/{random_seed}'
    if not os.path.exists(dir_save):
        os.mkdir(dir_save)
    pipeline_result.save_to_directory(dir_save)

    db.add(f'model_{random_seed}', pipeline_result.model)

INFO:pykeen.datasets.utils:Loading cached preprocessed dataset from file:///home/acg16558pn/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM
INFO:pykeen.triples.triples_factory:Loading from file:///home/acg16558pn/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/training
INFO:pykeen.triples.triples_factory:Loading from file:///home/acg16558pn/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/testing
INFO:pykeen.triples.triples_factory:Loading from file:///home/acg16558pn/.data/pykeen/datasets/fb15k237/cache/47DEQpj8HBSa-_TImW-5JCeuQeRkm5NM/validation
INFO:pykeen.pipeline.api:Using device: None


--------------------
learn knoweldge graph embedding with random seed 0


INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: /home/acg16558pn/.data/pykeen/checkpoints/best-model-weights-f7ee1015-bcf6-4729-a81f-02f561d5ab58.pt
INFO:pykeen.triples.triples_factory:Creating inverse triples.


Training epochs on cuda:0:   0%|          | 0/201 [00:00<?, ?epoch/s]

INFO:pykeen.triples.triples_factory:Creating inverse triples.


...

Training batches on cuda:0:   0%|          | 0/2126 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/2126 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/2126 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/2126 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/2126 [00:00<?, ?batch/s]

In [ ]:
db.add('model_name', model_name)
db.add('dataset_name', dataset_name)
db.add('dict_args', dict_args)
db.add('f_params', f_params)
db.add('list_random_seeds', list_random_seeds)